In [21]:
import sys
import numpy as np
import keras
import os
from keras import models,layers
import random

In [10]:
path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt'
)
text = open(path , encoding= "utf-8").read().lower()

chars = sorted(list(set(text)))
char_indices = {c : i  for i , c in enumerate(chars)}
indices_char = {i : c  for i , c in enumerate(chars)}



In [ ]:
step  = 3
maxlen = 60
sentences = []
next_chars =  []
for i in range(0 , len(text) - maxlen ,step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])



In [12]:
x = np.zeros((len(sentences) , maxlen  , len(chars)) , dtype=bool)
y = np.zeros((len(sentences) , len(chars)) , dtype=bool)

In [13]:
for i , sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i ,t , char_indices[char]] = 1
    y[i , char_indices[next_chars[i]]] = 1

In [22]:
os.makedirs('modellolinguistico' , exist_ok=True)

In [ ]:
callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_accuracy' , patience = 1 ), 
                  keras.callbacks.ModelCheckpoint(filepath='models_keras.h5', monitor = "val_loss" , save_best_only=True),
                  keras.callbacks.TensorBoard(log_dir='modellolinguistico',histogram_freq=1 , embeddings_freq=1)]

In [26]:
model = models.Sequential([
    layers.Input(shape=(maxlen  , len(chars))),
    layers.LSTM(128 , dropout=0.2 , recurrent_dropout=0.2, return_sequences=True),
    layers.LSTM(128),
    layers.Dense(len(chars)  , activation='softmax')
])
optimizers = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'] , optimizer=optimizers)
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                   │ (None, 60, 128)        │        95,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 57)             │         7,353 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 234,169 (914.72 KB)

 Trainable params: 234,169 (914.72 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
def sample(preds , temperature = 1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(np.clip(preds , 1e-8,1)) / temperature
    exp_pred = np.exp(preds)
    preds = exp_pred / np.sum(exp_pred)
    probas = np.random.multinomial(1 , preds , 1)
    return np.argmax(probas)


In [ ]:
for epoch in range(1 , 61):
    model.fit(x,  y , batch_size=128 , epochs=1 , callbacks=callbacks_list)
    start = random.randint(0 , len(text) - maxlen - 1)
    for diversity in [0.2 , 0.5 , 1.0 , 1.2]:
        print(f"-----diversity: {diversity}")
        generated = ''
        generated += text[start :start + maxlen]
        print("-----Sentence: {}".format(sentence))
        sys.stdout.write(generated)
        for i in range(400):
            x_pred = np.zeros((1 , maxlen , len(chars)))
            for t , char in enumerate(sentence):
                x_pred[0 , t, char_indices[char]] = 1
            preds = model.predict(x_pred , verbose=0)[0]
            next_index = sample(preds , diversity)
            next_char = indices_char[next_index]
            generated += next_char
            sentence   = sentence[1:] + next_char
            sys.stdout.write(next_char)
            sys.stdout.flush()



1565/1565 ━━━━━━━━━━━━━━━━━━━━ 167s 105ms/step - accuracy: 0.2732 - loss: 2.5635


c:\Users\salva\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\callbacks\early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_accuracy` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)
c:\Users\salva\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\callbacks\model_checkpoint.py:209: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


-----diversity: 0.2
-----Sentence: t of the most be like of the something the gare have been ev
soporific
appliances--and that "virtue," in my opinion, has ery he diser and the fen and for the fene the can and his a man the fare the senser and sensing and the fine and and the preally and the prade and the every and the belight that the beling and the proper of the fare the man and the for the preser the bele of the preally the hister of the ending and the can the fare and will sen and the delief and such of the belight that the prefer the preser and -----diversity: 0.5
-----Sentence: lief and such of the belight that the prefer the preser and 
soporific
appliances--and that "virtue," in my opinion, has disprension and in chase that the delality of the preed that the ord that our be the dast and the pread of the mast that as san and from which all proate and the world for the erived to at orleine and the lang for the yen and digre and dight of the wisen and un and for a all from will mo